In [5]:
import heapq
import math

class Chamada:
    def __init__(self, id, local, severidade, tipo_vegetacao, clima):
        self.id = id
        self.local = local
        self.severidade = severidade
        self.tipo_vegetacao = tipo_vegetacao
        self.clima = clima

    def __repr__(self):
        return f"Chamada(id={self.id}, local={self.local}, severidade={self.severidade}, tipo_vegetacao={self.tipo_vegetacao}, clima={self.clima})"


class Equipe:
    def __init__(self, nome):
        self.nome = nome

    def __repr__(self):
        return f"Equipe({self.nome})"


class Area:
    def __init__(self, nome, status):
        self.nome = nome
        self.status = status  # 'ativo', 'contido', 'resolvido'

    def __repr__(self):
        return f"Area(nome={self.nome}, status={self.status})"


# Função para calcular a prioridade de uma chamada
def calcular_prioridade(chamada, peso_vegetacao):
    prioridade = chamada.severidade * peso_vegetacao.get(chamada.tipo_vegetacao, 1)
    return prioridade


# Função para calcular o menor caminho usando o algoritmo de Dijkstra
def calcular_menor_caminho(grafo, origem, destino):
    dist = {origem: 0}
    prev = {origem: None}
    heap = [(0, origem)]
    
    while heap:
        (custo, atual) = heapq.heappop(heap)
        
        if atual == destino:
            caminho = []
            while prev[atual] is not None:
                caminho.insert(0, atual)
                atual = prev[atual]
            return caminho
        
        for vizinho, peso in grafo.get(atual, {}).items():
            novo_custo = custo + peso
            if vizinho not in dist or novo_custo < dist[vizinho]:
                dist[vizinho] = novo_custo
                prev[vizinho] = atual
                heapq.heappush(heap, (novo_custo, vizinho))

    return []  # Caso não encontre caminho


# Função para registrar ações de uma equipe (Pilha)
def registrar_acao(pilha, acao):
    pilha.append(acao)


# Função para atender a chamada, considerando a prioridade, rota e ações
def atender_chamada(chamada, grafo, equipes, peso_vegetacao, pilha, lista_areas):
    prioridade = calcular_prioridade(chamada, peso_vegetacao)

    # Definir rota (caminho mais curto)
    origem = "Base Central"  
    destino = chamada.local
    rota = calcular_menor_caminho(grafo, origem, destino)

    # Definir ações baseadas na severidade e tipo de vegetação
    acao = []
    if chamada.severidade >= 5:
        acao.append("Aplicar barreira de contenção")
        acao.append("Criar aceiro")
    elif chamada.severidade >= 3:
        acao.append("Monitorar foco")
    else:
        acao.append("Reforçar vigilância")

    # Registrar ações na pilha
    for a in acao:
        registrar_acao(pilha, a)

    # Atualizar o status da área afetada
    for area in lista_areas:
        if area.nome == chamada.local:
            area.status = "controle em andamento"
    
    # Simular atendimento (usando a equipe disponível, por exemplo)
    equipe = equipes[0]  # Assume que a equipe [0] é a primeira disponível
    equipes.remove(equipe)  # Retira a equipe da lista de disponíveis

    return {
        "ocorrencia_id": chamada.id,
        "prioridade": prioridade,
        "acao": acao,
        "rota": rota,
        "tempo_estimado": sum(grafo[rota[i]].get(rota[i + 1], 0) for i in range(len(rota) - 1)),
        "status_area": lista_areas[0].status
    }


In [4]:
# Entrada e dados
peso_vegetacao = {
    "cerrado": 1.2,
    "mata_atlantica": 1.5,
    "pantanal": 2.0
}

grafo = {
    "Base Central": {"Zona Norte": 10, "Vila Verde": 5},
    "Zona Norte": {"Base Central": 10, "Mata Alta": 7},
    "Vila Verde": {"Base Central": 5, "Mata Alta": 3},
    "Mata Alta": {"Zona Norte": 7, "Vila Verde": 3}
}

chamadas = [
    Chamada(id=1, local="Zona Norte", severidade=4, tipo_vegetacao="cerrado", clima="seco"),
    Chamada(id=2, local="Mata Alta", severidade=5, tipo_vegetacao="pantanal", clima="seco")
]

equipes = [Equipe(nome="Equipe A"), Equipe(nome="Equipe B")]
pilha = []
lista_areas = [Area(nome="Zona Norte", status="ativo"), Area(nome="Mata Alta", status="ativo")]



In [3]:
# Atendimento das chamadas
for chamada in chamadas:
    resultado = atender_chamada(chamada, grafo, equipes, peso_vegetacao, pilha, lista_areas)
    print(resultado)


{'ocorrencia_id': 1, 'prioridade': 4.8, 'acao': ['Monitorar foco'], 'rota': ['Zona Norte'], 'tempo_estimado': 0, 'status_area': 'controle em andamento'}
{'ocorrencia_id': 2, 'prioridade': 10.0, 'acao': ['Aplicar barreira de contenção', 'Criar aceiro'], 'rota': ['Vila Verde', 'Mata Alta'], 'tempo_estimado': 3, 'status_area': 'controle em andamento'}
